In [1]:
from dataset import StarCraftMotionDataset, pad_collate_fn
from torch.utils.data import DataLoader, Dataset
import torch
import torch_scatter
import torch.nn as nn

criterion = nn.MSELoss(reduction='sum')

batch_size = 16

test_dataset = StarCraftMotionDataset("/local/scratch/a/bai116/datasets/StarCraftMotion_v0.2/processed", train=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, collate_fn=pad_collate_fn)

In [2]:
col = torch.tensor([])

print(r"position  velocity   super")
print(len(test_loader))
for i, data in enumerate(test_loader):
    if i > 1000:
        break
    mask = torch.ones(data.shape, dtype=torch.bool)
    
    diff = torch.abs(data[:,:,1:,2] - data[:,:,:-1,2] + data[:,:,1:,3] - data[:,:,:-1,3])
    small_diff = torch.nonzero(diff < 10)
    
    mask[small_diff[:,0],small_diff[:,1]] = False
    

position  velocity   super
27741


In [17]:
col.to(int).bincount()  [20,36,75]

tensor([  0,   0,  41,   0,   1,   0,   0,   0,   0,   0,   0,   1,   0,   0,
          0,   0,   0,   0,   0, 107,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,  88,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   4,   0,   0,   0,   0,   0,   0,   0,   8,   0,   0,
          0,   2,   0,   0,   0,   0,   0,   0,   1,   0,   0,   0,   2,   4,
          0,   0,   0,  15, 168,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   1,   0,   0,   0,   0,   0,   4,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   6,   0,   2,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   8,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   1,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   3,   0,   0,   0,   0,   9,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,  

In [108]:
print(test_dataset[37].shape)

torch.Size([1334, 6, 13])


In [63]:
print(data[5,:,59,:])

tensor([[180.,   2., 213., 210., 169.,   2., 101., 208., 255.,   1.,   1., 211.,
         206.],
        [180.,   2., 213., 210., 169.,   2., 101., 208., 255.,   1.,   1., 211.,
         206.],
        [180.,   2., 213., 210., 169.,   2., 101., 208., 255.,   1.,   1., 211.,
         206.],
        [180.,   2., 213., 210., 169.,   2., 101., 208., 255.,   1.,   1., 211.,
         206.],
        [180.,   2., 240., 104., 127.,   2., 136., 255., 255.,   1.,   1., 211.,
         206.],
        [180.,   2., 240., 104., 127.,   2., 136., 255., 255.,   1.,   1., 211.,
         206.]])


In [73]:
print(data[7,:,132,:])

tensor([[ 19.,   2.,  37., 178., 185.,   3.,  77.,  95., 255.,   0.,   0.,   0.,
           0.],
        [ 19.,   2.,  37., 178., 185.,   3.,  77.,  95., 255.,   0.,   0.,   0.,
           0.],
        [ 19.,   2.,  47., 120., 186.,   3., 137.,  95., 255.,   0.,   0.,   0.,
           0.],
        [ 19.,   2.,  48., 117., 192.,   3., 137.,  95., 255.,   0.,   0.,   0.,
           0.],
        [ 19.,   2.,  48., 114., 196.,   3., 111.,  95., 255.,   0.,   0.,   0.,
           0.],
        [ 19.,   2.,  46., 112., 197.,   3.,  82.,  95., 255.,   0.,   0.,   0.,
           0.]])


In [23]:
import torch

# Example tensor with shape (num_agent, time_frame, attributes)
data = torch.randn(6, 4, 8)  # Example dimensions: time_frame=10, num_agent=20, attributes=4

mask = data[:,0,7] != 0
print(mask.shape)
# Mask tensor with shape (N, 3), where each row corresponds to (batch_index, time_frame_index, agent_index)
mask_indices = torch.tensor([[1, 3], [2, 2], [1, 1]])  # Example mask indices
print(mask_indices[:,0])

# Update the mask to False for agents to be filtered out
mask[mask_indices[:, 0]] = False
print(mask.shape)
print(data[mask].shape)
# Apply the mask along the `num_agent` dimension
filtered_data = data[mask].reshape(-1, time_frame, attributes)  # Reshape to maintain structure

torch.Size([6])
tensor([1, 2, 1])
torch.Size([6])
torch.Size([4, 4, 8])


In [19]:
print(filtered_data.shape)

torch.Size([4, 4, 8])


In [20]:
print(data)

tensor([[[ 0.2818,  0.0515,  0.1822,  0.2732, -0.6128,  0.0080, -0.2544,
           1.0446],
         [-0.3375, -0.4980, -0.6375, -1.0652, -0.0873, -0.1038, -0.5079,
           0.0122],
         [-0.0262, -0.0511,  0.4583, -0.5820,  0.3686,  1.6570,  0.1492,
           0.9172],
         [-0.0440, -1.1786,  0.0336,  0.6290,  1.8033, -0.4377,  1.1811,
           0.0085]],

        [[-0.7107, -0.2221, -2.0404, -0.2333, -1.0106, -0.7140, -1.0726,
           0.0919],
         [-1.7272, -1.2012,  0.0701,  1.7531,  0.7815,  0.9002,  0.4174,
          -0.1821],
         [ 1.8921, -0.1599,  0.5901, -1.0917, -2.0166,  0.5787,  0.3665,
          -0.0412],
         [ 0.0162, -0.8904,  0.2815,  1.4737,  0.3270, -0.1670,  0.0399,
          -0.5112]],

        [[ 0.1791, -0.3445,  1.9330,  0.2140,  1.9267, -0.2385, -0.6469,
           0.0615],
         [ 1.5100, -0.4589, -0.5386, -0.6384,  0.2739, -1.3766, -0.8195,
           1.2362],
         [-0.3095, -0.4454, -0.9137,  0.9998,  0.6851,  0.3212,  0

In [21]:
print(filtered_data)

tensor([[[ 0.2818,  0.0515,  0.1822,  0.2732, -0.6128,  0.0080, -0.2544,
           1.0446],
         [-0.3375, -0.4980, -0.6375, -1.0652, -0.0873, -0.1038, -0.5079,
           0.0122],
         [-0.0262, -0.0511,  0.4583, -0.5820,  0.3686,  1.6570,  0.1492,
           0.9172],
         [-0.0440, -1.1786,  0.0336,  0.6290,  1.8033, -0.4377,  1.1811,
           0.0085]],

        [[ 1.5747, -0.2960,  0.4736, -1.4567,  0.4268, -0.0233,  0.6718,
          -0.8725],
         [-0.4801,  0.8368, -1.3065,  0.9219,  1.3431,  0.4575,  0.5764,
          -1.7702],
         [ 2.4335,  0.3968, -0.2304,  0.9427,  1.5273, -0.5563,  0.0323,
           2.0987],
         [ 0.3972,  0.0133,  1.2845, -0.2488, -0.4270, -1.8521,  1.3313,
          -1.2706]],

        [[ 0.4905,  0.4385, -0.2905, -1.5343,  0.3931,  0.9804, -0.9868,
           0.5464],
         [ 1.1006,  0.1590, -0.7542,  0.2978,  2.4357, -1.9707, -0.6703,
          -0.7943],
         [-3.4836, -0.2588, -0.8260, -0.6465, -1.4242, -0.3508, -0